# Test code markdown

In [558]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [559]:
df = pd.read_csv(r'files\clean.csv')

# Garder que les variables quanti
variables_quali = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country','income']
df = df.drop(columns= variables_quali)
df

,age,capital-gain,capital-loss,hours-per-week
0,25,0,0,40
1,38,0,0,50
2,28,0,0,40
3,44,7688,0,40
4,34,0,0,30
...,...,...,...,...
45217,27,0,0,38
45218,40,0,0,40
45219,58,0,0,40
45220,22,0,0,20


In [560]:
#Tire une clef de repartition aleatoire de % d'individues dans les classes

def clef_repartition_random(nb_classe):
    clef_repartition = [10] * nb_classe
    somme_restant = 100 - sum(clef_repartition)

    for i in range(nb_classe):
        if somme_restant <= 0:
            break
        max_val = somme_restant + clef_repartition[i]
        nouvelle_valeur = random.randint(10, max_val)
        somme_restant -= (nouvelle_valeur - clef_repartition[i])
        clef_repartition[i] = nouvelle_valeur
    
    clef_repartition[-1] += somme_restant
    random.shuffle(clef_repartition)
    
    return clef_repartition

In [561]:
def repartition_individues(clef_repartition, taille_df) :
    n_repartition = [round(taille_df * x / 100) for x in clef_repartition]

    if sum(n_repartition) > taille_df :
        n_repartition[-1] -= 1
    
    if sum(n_repartition) < taille_df :
        n_repartition[-1] += 1

    return n_repartition

print(len(df))
for i in range (0,100) :
    repartition_finale = repartition_individues(clef_repartition_random(5), len(df))
    print(repartition_finale,sum(repartition_finale))

45222
[4974, 4522, 16732, 14019, 4975] 45222
[5879, 11758, 5879, 8592, 13114] 45222
[9044, 4974, 12662, 4974, 13568] 45222
[18089, 4522, 7236, 10853, 4522] 45222
[11306, 4522, 4522, 15828, 9044] 45222
[20802, 4522, 10401, 4974, 4523] 45222
[4974, 5427, 4522, 4522, 25777] 45222
[5427, 4522, 25324, 5427, 4522] 45222
[4522, 4974, 4974, 25777, 4975] 45222
[6331, 5427, 5427, 4522, 23515] 45222
[7236, 9044, 14019, 7688, 7235] 45222
[4974, 6783, 14471, 6331, 12663] 45222
[14471, 7236, 6331, 4522, 12662] 45222
[4522, 26681, 4522, 4522, 4975] 45222
[4522, 16280, 5879, 10853, 7688] 45222
[4974, 17637, 8140, 4522, 9949] 45222
[11758, 5879, 14019, 7688, 5878] 45222
[5879, 4522, 25777, 4522, 4522] 45222
[5427, 4522, 12662, 5879, 16732] 45222
[9497, 5879, 13114, 9949, 6783] 45222
[4522, 7236, 5427, 4522, 23515] 45222
[14923, 6331, 4974, 10853, 8141] 45222
[20802, 7688, 6331, 4974, 5427] 45222
[19898, 10401, 4522, 4974, 5427] 45222
[6331, 14019, 7236, 12210, 5426] 45222
[6783, 5427, 5427, 11758, 1582